## Importing Dependencies

In [1]:
import tensorflow as tf
import os
import cv2
from matplotlib import pyplot as plt
import numpy as np

from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
#Using GPU with nvidia cuda to enhance tensorflow
#I use this just to increase my execution time. The model can be trained even without this cell.

gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

## Loading and pre-processing data

In [3]:
classes = os.listdir('data/train_angle_2')

In [4]:
classes

['Back', 'Front', 'Side']

In [5]:
#use Imagedatagenerator
datagen = tf.keras.preprocessing.image.ImageDataGenerator(
        rescale=1./255,
        validation_split=0.2)


train_generator = datagen.flow_from_directory(
        'data/train_angle_2',
        target_size=(250, 250),
        batch_size=32,
        class_mode='categorical',
        subset='training')  # This will be used for training

validation_generator = datagen.flow_from_directory(
        'data/train_angle_2',
        target_size=(250, 250),
        batch_size=32,
        class_mode='categorical',
        subset='validation')



Found 1008 images belonging to 3 classes.
Found 251 images belonging to 3 classes.


## Creating model with convolution layers

In [6]:
no_of_classes = 3

model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(250,250, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Flatten(name='Flatten'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(256, activation='relu', name='Dense'),
    tf.keras.layers.Dense(no_of_classes, activation='softmax', name='Dense_finalLayer')
])

print(model.summary())

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])



Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 248, 248, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 124, 124, 32)     0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 124, 124, 32)      0         
                                                                 
 conv2d_1 (Conv2D)           (None, 122, 122, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 61, 61, 64)       0         
 2D)                                                             
                                                                 
 dropout_1 (Dropout)         (None, 61, 61, 64)        0

## Training the model

In [7]:
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

history = model.fit(train_generator, validation_data=validation_generator, epochs=50,
                    steps_per_epoch=len(train_generator), callbacks=[early_stopping])

Epoch 1/50
11/32 [=========>....................] - ETA: 11s - loss: 1.6311 - accuracy: 0.3693

C:\Users\Nikesh\AppData\Local\Programs\Python\Python39\lib\site-packages\PIL\Image.py:996: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


32/32 [==============================] - 27s 744ms/step - loss: 1.2634 - accuracy: 0.4167 - val_loss: 1.0753 - val_accuracy: 0.4622
Epoch 2/50
32/32 [==============================] - 19s 583ms/step - loss: 0.9640 - accuracy: 0.5218 - val_loss: 1.0332 - val_accuracy: 0.4343
Epoch 3/50
32/32 [==============================] - 18s 562ms/step - loss: 0.8385 - accuracy: 0.6081 - val_loss: 0.8002 - val_accuracy: 0.6096
Epoch 4/50
32/32 [==============================] - 18s 554ms/step - loss: 0.5021 - accuracy: 0.7927 - val_loss: 0.5178 - val_accuracy: 0.7769
Epoch 5/50
32/32 [==============================] - 18s 563ms/step - loss: 0.3555 - accuracy: 0.8591 - val_loss: 0.4671 - val_accuracy: 0.7012
Epoch 6/50
32/32 [==============================] - 18s 557ms/step - loss: 0.2158 - accuracy: 0.9167 - val_loss: 0.2698 - val_accuracy: 0.9044
Epoch 7/50
32/32 [==============================] - 18s 557ms/step - loss: 0.1496 - accuracy: 0.9514 - val_loss: 0.4741 - val_accuracy: 0.8606
Epoch 8/50

## Validation

C:\Users\Nikesh\AppData\Local\Temp\ipykernel_23928\2085538597.py:9: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  resized = img_rgba.resize((250, 250), Image.ANTIALIAS)


In [14]:
img = cv2.imread('data/train_angle_2/Front/TESTS.jpg')


from PIL import Image
img_pil = Image.fromarray(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
img_rgba = img_pil.convert('RGBA')

# Resize the image to the required input size
resized = img_rgba.resize((250, 250), Image.ANTIALIAS)

# Convert the image back to RGB format (if needed for displaying with matplotlib)
resized_rgb = resized.convert('RGB')
plt.imshow(np.array(resized_rgb))
plt.show()



In [15]:
resized = np.array(resized_rgb) / 255.0  # Convert to float and rescale to [0, 1]
resized = np.expand_dims(resized, 0)  # Add batch dimension
resized_tensor = tf.convert_to_tensor(resized, dtype=tf.float32)

test_predict = model.predict(resized_tensor)

# Get the predicted class label
predicted_class_index = np.argmax(test_predict[0])
predicted_class = classes[predicted_class_index]
print('Predicted Car:', predicted_class)

1/1 [==============================] - 0s 211ms/step
Predicted Car: Front


## Saving model and converting as tflite model

In [10]:
#model.save('angle_detection_model.h5')

In [11]:
#from keras.models import load_model
#angle_detection_model = load_model('angle_detection_model.h5')

In [12]:
#conv = tf.lite.TFLiteConverter.from_keras_model(angle_detection_model)
#tfmodel = conv.convert()

INFO:tensorflow:Assets written to: C:\Users\Nikesh\AppData\Local\Temp\tmp0op1d1i7\assets


INFO:tensorflow:Assets written to: C:\Users\Nikesh\AppData\Local\Temp\tmp0op1d1i7\assets


In [13]:
#open("angle_detection_model_lite.tflite","wb").write(tfmodel)

22680804